In [1]:
# set workspace
WhoIsWorking = "RT"
WorkLocation = "School"

if WhoIsWorking == "RT":
    if WorkLocation == "Home":
        folder_path = r'C:\Users\rentr\Desktop\ENGO\DataMining\ENGO645_TermProject_Data'
    elif WorkLocation == "School":
        folder_path = r'D:\ENGO645_TermProject_Data'
elif WhoIsWorking == "AA":
    folder_path = ''
elif WhoIsWorking == "IA":
    folder_path = ''
elif WhoIsWorking == "AI":
    folder_path = ''

print(folder_path)

D:\ENGO645_TermProject_Data


In [2]:
import os

# List all files in the folder
files = os.listdir(folder_path)

# Print the list of files
for file in files:
    print(file)

BuildingOutlines.CPG
BuildingOutlines.dbf
BuildingOutlines.prj
BuildingOutlines.sbn
BuildingOutlines.sbx
BuildingOutlines.shp
BuildingOutlines.shp.xml
BuildingOutlines.shx
CensusProfile_2021_PopAndIncome_Reformatted.csv
ClimateData_Hourly.csv
EnhancedPointsOfInterestPoint_Toronto.cpg
EnhancedPointsOfInterestPoint_Toronto.dbf
EnhancedPointsOfInterestPoint_Toronto.prj
EnhancedPointsOfInterestPoint_Toronto.sbn
EnhancedPointsOfInterestPoint_Toronto.sbx
EnhancedPointsOfInterestPoint_Toronto.shp
EnhancedPointsOfInterestPoint_Toronto.shp.xml
EnhancedPointsOfInterestPoint_Toronto.shx
HousePrices_Condos.csv
HousePrices_Detached.csv
HousePrices_SemiDetached.csv
LandUseClassified_20211220.cpg
LandUseClassified_20211220.dbf
LandUseClassified_20211220.prj
LandUseClassified_20211220.sbn
LandUseClassified_20211220.sbx
LandUseClassified_20211220.shp
LandUseClassified_20211220.shp.xml
LandUseClassified_20211220.shx
MajorCrimeIndicators.csv
Neighbourhoods.CPG
Neighbourhoods.dbf
Neighbourhoods.prj
Neighb

In [3]:
import os
import geopandas as gpd

# Path to the shapefile for neighborhoods
neighborhoods_shapefile_path = os.path.join(folder_path, 'Neighbourhoods.shp')

# Read the shapefile into a GeoDataFrame
neighborhoods_gdf = gpd.read_file(neighborhoods_shapefile_path)

# Print the first few lines of the  table
neighborhoods_gdf.head(2)

,_id1,AREA_ID2,AREA_AT3,PARENT_4,AREA_SH5,AREA_LO6,AREA_NA7,AREA_DE8,CLASSIF9,CLASSIF10,OBJECTI11,geometry
0,1,2502366,26022881,0,174,174,South Eglinton-Davisville,South Eglinton-Davisville (174),Not an NIA or Emerging Neighbourhood,NA,17824737.0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697..."
1,2,2502365,26022880,0,173,173,North Toronto,North Toronto (173),Not an NIA or Emerging Neighbourhood,NA,17824753.0,"POLYGON ((-79.39744 43.70693, -79.39837 43.706..."


In [4]:
# Keep only certain fields and rename fields
neighborhoods_gdf = neighborhoods_gdf[['geometry', '_id1', 'AREA_NA7']].rename(columns={'_id1': 'UID', 'AREA_NA7': 'NeighbourhoodName'})

# Print the first few lines of the updated GeoDataFrame
neighborhoods_gdf.head(2)

,geometry,UID,NeighbourhoodName
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto


In [5]:
import geopandas as gpd
import os

# Path to the shapefile for poles
poles_shapefile_path = os.path.join(folder_path, 'Poles.shp')

# Read shapefiles into GeoDataFrames
poles_gdf = gpd.read_file(poles_shapefile_path)

# List fields in the poles GeoDataFrame
poles_fields = poles_gdf.columns
print("Fields in the poles GeoDataFrame:")
print(poles_fields)

Fields in the poles GeoDataFrame:
Index(['objectid', 'subtype', 'subtypecod', 'elevation', 'X', 'Y', 'geometry'], dtype='object')


In [6]:
# List unique attributes in the 'subtype' field of poles
unique_subtypes = poles_gdf['subtype'].unique()

# Print unique attributes
print("Unique attributes in the 'subtype' field of poles:")
print(unique_subtypes)

Unique attributes in the 'subtype' field of poles:
['Miscellaneous Pole' 'Street Light and Traffic Signal Pole'
 'Traffic Signal Pole' 'Street Light Pole' 'Pedestrian Light Pole'
 'Circular Hydro Tower']


In [7]:
# Filter poles to only keep certain subtypes
desired_subtypes = ['Street Light and Traffic Signal Pole', 'Street Light Pole', 'Pedestrian Light Pole']
filtered_poles_gdf = poles_gdf[poles_gdf['subtype'].isin(desired_subtypes)]

In [8]:
# Perform spatial join to count the number of poles in each neighborhood
poles_per_neighborhood = gpd.sjoin(neighborhoods_gdf, filtered_poles_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of poles as a new column to the neighborhoods GeoDataFrame
neighborhoods_gdf['Poles_Count'] = neighborhoods_gdf['NeighbourhoodName'].map(poles_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighborhoods_gdf.head(2)

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UID,NeighbourhoodName,Poles_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286


In [9]:
# Path to the shapefile for POI
poi_shapefile_path = os.path.join(folder_path, 'EnhancedPointsOfInterestPoint_Toronto.shp')

# Read the shapefile into a GeoDataFrame
poi_gdf = gpd.read_file(poi_shapefile_path)

# Perform spatial join to count the number of POIs within each neighbourhood
poi_per_neighborhood = gpd.sjoin(neighborhoods_gdf, poi_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of POIs as a new column to the neighborhoods GeoDataFrame
neighborhoods_gdf['POI_Count'] = neighborhoods_gdf['NeighbourhoodName'].map(poi_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighborhoods_gdf.head()

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281


In [10]:
# Path to the shapefile for traffic cameras
traffic_cameras_shapefile_path = os.path.join(folder_path, 'TrafficCamera.shp')

# Read the shapefile into a GeoDataFrame
traffic_cameras_gdf = gpd.read_file(traffic_cameras_shapefile_path)

# Perform spatial join to count the number of traffic cameras within each neighborhood
traffic_cameras_per_neighborhood = gpd.sjoin(neighborhoods_gdf, traffic_cameras_gdf, how="left", op="contains").groupby('NeighbourhoodName').size()

# Add the count of traffic cameras as a new column to the neighborhoods GeoDataFrame
neighborhoods_gdf['TCamera_Count'] = neighborhoods_gdf['NeighbourhoodName'].map(traffic_cameras_per_neighborhood)

# Display the first few rows of the updated neighborhoods GeoDataFrame
neighborhoods_gdf.head()

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3493: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


,geometry,UID,NeighbourhoodName,Poles_Count,POI_Count,TCamera_Count
0,"POLYGON ((-79.38635 43.69783, -79.38623 43.697...",1,South Eglinton-Davisville,649,1185,2
1,"POLYGON ((-79.39744 43.70693, -79.39837 43.706...",2,North Toronto,286,851,2
2,"POLYGON ((-79.43411 43.66015, -79.43537 43.659...",3,Dovercourt Village,1170,534,1
3,"POLYGON ((-79.43870 43.66766, -79.43841 43.666...",4,Junction-Wallace Emerson,1663,876,1
4,"POLYGON ((-79.38404 43.64497, -79.38502 43.644...",5,Yonge-Bay Corridor,1139,20281,12
